In [1]:
from random import choice
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [5]:
# Search space saves all the possibe choices
# OBJECTS_PARENTS help us identify which arguments belongs to which object
SEARCH_SPACE = []
OBJECTS_PARENTS = {}

In [6]:
# Abstraction of the normal objects that can save its arguments in a list
# The symbolic object help to recreate the original object with different arguments
class symbolic_object:
    def __init__(self):
        self.args = []
        self.name = None
        self.activation = None
        
    # This method save the relationship between the object (parent) and the arguments
    def __call__(self, *args, activation = None):
        self.args = args
            
        for i in range(len(self.args)):
            OBJECTS_PARENTS[id(self.args[i])]= id(self)
            
        if(activation is not None):
            self.activation = activation
            OBJECTS_PARENTS[id(activation)] = id(self)
            
        return self
    
    def set_name(self, name):
        self.name = name
        self.__name__ = 'symbolic'
        
    def read_name(self):
        print(self.name)
        
    def get_args(self):
        arg_list = []
        for i in range(len(self.args)):
            arg_list.append(self.args[i])
        
        if(self.activation is not None):
            arg_list.append(self.activation)
        return arg_list

In [3]:
# Takes a class as an agrument and returns a symbolic object 
def symbolize(real_object):
    name = real_object.__name__   
    sy_obj = symbolic_object()
    sy_obj.set_name(name)
    return sy_obj

In [4]:
# takes a class name, the wanted agruments and the number of arguments and returns an executable object
# Helpful method for the materlize method
def create_object_from_symbols(class_name, arguments, args_num):
    temp_dict = {}
    if (args_num == 0):
        mycode = "temp_dict['var']"+ ' = ' + class_name+'()'
        
    elif (args_num == 1):
        arguments = str(arguments[0])
        mycode = "temp_dict['var']"+ ' = ' + class_name+'('+arguments+')'
    
    else:
        args = str(arguments[0])
        for i in range(1, args_num):
            arguments[i] = str(arguments[i])
            args += ', ' +arguments[i]
        mycode = "temp_dict['var']"+ ' = ' + class_name+'('+args+')'
    exec(mycode)
    return temp_dict['var']

In [7]:
# totally random 
# returns indices of the choices
def random_search():
    choices = [None] * len(SEARCH_SPACE)
    number_of_choices = len(SEARCH_SPACE)
    for i in range(number_of_choices):
        indices = range(len(SEARCH_SPACE[i]))
        choices[i] = choice(indices)
    return choices